# Sub Question Query Engine - LlamaIndex + KDB.ai

## Install requirements

**Install required packages**

In [1]:
# !pip install llama-index llama-index-embeddings-huggingface llama-index-llms-openai llama-index-readers-file llama-index-vector-stores-kdbai
# !pip install kdbai_client sentence-transformers


[notice] A new release of pip available: 22.2.1 -> 24.1.2
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.1.2
[notice] To update, run: python3.10 -m pip install --upgrade pip


**Helper Library - To allow nested loop events**

In [2]:
import nest_asyncio

nest_asyncio.apply()

## Downloading data

**Libraries**

In [3]:
import os
import urllib.request

**Data directories and paths**

In [4]:
# Root path
root_path = os.path.abspath(os.getcwd())

# Data directory and path
data_dir = "data"
data_path = os.path.join(root_path, data_dir)
if not os.path.exists(data_path):
    os.mkdir(data_path)

**Downloading text**

In [5]:
text_url = "https://raw.githubusercontent.com/KxSystems/kdbai-samples/main/retrieval_augmented_generation/data/state_of_the_union.txt"
with urllib.request.urlopen(text_url) as response:
    text_content = response.read().decode("utf-8")

text_file_name = text_url.split('/')[-1]
text_path = os.path.join(data_path, text_file_name)
if not os.path.exists(text_path):
    with open(text_path, 'w') as text_file:
        text_file.write(text_content)

metadata = {
    f"{data_dir}/{text_file_name}": {
        "title": text_file_name.split('.')[0],
        "file_path": text_path
    }
}

**Show text data**

In [6]:
def show_text(text_path):
    if os.path.isfile(text_path):
        with open(text_path, 'r') as text_file:
            contents = text_file.read()
        print(contents[:512])
        print("="*80)

In [7]:
show_text(text_path)

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russi


## KDB.ai Vector Database - session and tables

**Libraries**

In [8]:
import kdbai_client as kdbai

**KDB.ai session**

In [9]:
KDBAI_ENDPOINT = "http://localhost:8085"
session = kdbai.Session(endpoint=KDBAI_ENDPOINT)

**KDB.ai table**

In [10]:
# Table - name & schema
table_name = "sqqe_docs"
table_schema = {
    "columns": [
        dict(name="document_id", pytype="bytes"),
        dict(name="text", pytype="bytes"),
        dict(
            name="embedding",
            vectorIndex=dict(type="flat", metric="L2", dims=768),
        ),
        dict(name="title", pytype="bytes"),
        dict(name="file_path", pytype="bytes")
    ]
}

In [11]:
# Drop table if exists
if table_name in session.list():
    session.table(table_name).drop()

In [12]:
# Create table
table = session.create_table(table_name, table_schema)

In [13]:
# Show table schema
table.schema()

{'columns': [{'name': 'document_id', 'qtype': 'string', 'pytype': 'bytes'},
  {'name': 'text', 'qtype': 'string', 'pytype': 'bytes'},
  {'name': 'embedding',
   'vectorIndex': {'type': 'flat', 'metric': 'L2', 'dims': 768},
   'qtype': 'reals',
   'pytype': 'float32'},
  {'name': 'title', 'qtype': 'string', 'pytype': 'bytes'},
  {'name': 'file_path', 'qtype': 'string', 'pytype': 'bytes'}]}

## Loading data

In [14]:
from llama_index.core import SimpleDirectoryReader

In [15]:
# Helper function - for getting metadata
def get_metadata(file_path):
    return metadata[file_path]

In [16]:
%%time

local_files = [fpath for fpath in metadata]
documents = SimpleDirectoryReader(input_files=local_files, file_metadata=get_metadata)

docs = documents.load_data()
len(docs)

CPU times: user 5.92 ms, sys: 8.51 ms, total: 14.4 ms
Wall time: 14.1 ms


1

## Creating Vector Store Index for data

**OpenAI API Key**

In [17]:
from getpass import getpass

In [18]:
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key: ")

OpenAI API key:  ········


**Text embeddings model**

In [19]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [20]:
EMBEDDING = "sentence-transformers/all-mpnet-base-v2"
embeddings_model = HuggingFaceEmbedding(model_name=EMBEDDING)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**LLM model**

In [21]:
from llama_index.llms.openai import OpenAI

In [22]:
LLM = "gpt-3.5-turbo"
llm_model = OpenAI(temperature=0, model=LLM)

**Setting callbacks and debug handler**

In [23]:
from llama_index.core.callbacks import LlamaDebugHandler
from llama_index.core.callbacks import CallbackManager

In [24]:
# Using the LlamaDebugHandler to print the trace of the sub questions captured by the SUB_QUESTION callback event type
llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

**Create vector store, storage context and the index for retrieval, query purposes**

In [25]:
from llama_index.vector_stores.kdbai import KDBAIVectorStore
from llama_index.core import StorageContext
from llama_index.core import Settings
from llama_index.core.indices import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter

In [26]:
%%time

# Vector Store
text_store = KDBAIVectorStore(table=table)

# Storage context
storage_context = StorageContext.from_defaults(vector_store=text_store)

# Settings
Settings.callback_manager = callback_manager
Settings.transformations = [SentenceSplitter(chunk_size=500, chunk_overlap=0)]
Settings.embed_model = embeddings_model
Settings.llm = llm_model

# Vector Store Index
index = VectorStoreIndex.from_documents(
    docs,
    use_async=True,
    storage_context=storage_context,
)

**********
Trace: index_construction
    |_embedding -> 1.796894 seconds
    |_embedding -> 1.806952 seconds
**********
CPU times: user 2.97 s, sys: 122 ms, total: 3.09 s
Wall time: 3.3 s


## Setup sub question query engine

**Index as Vector Query Engine**

In [27]:
# Vector query engine
vector_query_engine = index.as_query_engine()

**Setting up Sub Question Query Engine**

In [28]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

In [29]:
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="state_of_union",
            description="State of Union Speech",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=True,
)

**Querying the Sub Question Query Engine**

In [30]:
response = query_engine.query(
    "what did the president say about ukraine, what are the four common sense steps and how he planned to fight inflation?"
)

Generated 3 sub questions.
[state_of_union] Q: What did the president say about Ukraine?
[state_of_union] Q: What are the four common sense steps mentioned by the president?
[state_of_union] Q: How did the president plan to fight inflation?
[state_of_union] A: The four common sense steps mentioned by the president are staying protected with vaccines and treatments, preparing for new variants, ending the shutdown of schools and businesses, and continuing to vaccinate the world.
[state_of_union] A: The president planned to fight inflation by lowering costs instead of wages, increasing the productive capacity of the economy, making more goods in America, investing in infrastructure and innovation, creating more jobs with good wages, and reducing reliance on foreign supply chains.
[state_of_union] A: The president expressed support for Ukraine and its people, highlighting their fearlessness, courage, and determination in the face of aggression. He mentioned providing over $1 billion in dir

In [31]:
print(response)

The president expressed support for Ukraine and its people, highlighting their fearlessness, courage, and determination in the face of aggression. The four common sense steps mentioned by the president are staying protected with vaccines and treatments, preparing for new variants, ending the shutdown of schools and businesses, and continuing to vaccinate the world. The president planned to fight inflation by lowering costs instead of wages, increasing the productive capacity of the economy, making more goods in America, investing in infrastructure and innovation, creating more jobs with good wages, and reducing reliance on foreign supply chains.


**Iterate through all subquestions captured in SUB_QUESTION event**

In [32]:
from llama_index.core.callbacks import CBEventType, EventPayload

In [33]:
for i, (start_event, end_event) in enumerate(
    llama_debug.get_event_pairs(CBEventType.SUB_QUESTION)
):
    end_event_exception = end_event.payload.get(EventPayload.EXCEPTION)
    if end_event_exception is None:
        qa_pair = end_event.payload[EventPayload.SUB_QUESTION]
        print("Sub Question " + str(i) + ": " + qa_pair.sub_q.sub_question.strip())
        print("Answer: " + qa_pair.answer.strip())
        print("="*80)

Sub Question 0: What did the president say about Ukraine?
Answer: The president expressed support for Ukraine and its people, highlighting their fearlessness, courage, and determination in the face of aggression. He mentioned providing over $1 billion in direct assistance to Ukraine and emphasized that American forces are not engaged in conflict with Russian forces in Ukraine but are prepared to defend NATO Allies if necessary.
Sub Question 1: What are the four common sense steps mentioned by the president?
Answer: The four common sense steps mentioned by the president are staying protected with vaccines and treatments, preparing for new variants, ending the shutdown of schools and businesses, and continuing to vaccinate the world.
Sub Question 2: How did the president plan to fight inflation?
Answer: The president planned to fight inflation by lowering costs instead of wages, increasing the productive capacity of the economy, making more goods in America, investing in infrastructure a